In [ ]:
import nest_asyncio
from pydantic_ai import Agent, AgentRunResultEvent, AgentStreamEvent

from voice_agent_flow.llms import create_pydantic_azure_openai

nest_asyncio.apply()
import asyncio
from pprint import pprint

model = create_pydantic_azure_openai(model_name = "gpt-4.1")

agent = Agent(model)

**1. Message Types**

In [2]:
result_sync = agent.run_sync('What is the capital of Italy?')
print(result_sync.output)

The capital of Italy is **Rome**.


*Retrieve message history*   
1. A message is either a ModelRequest or a ModelResponse Message.   
2. ModelRequest and ModelResponse include a part parameter, which is a list of parts.   
3. Parts can be TextPart, ToolCallPart, ToolReturnPart, UserPromptPart and SystemPromptPart.   

In [3]:
result_sync.all_messages()

[ModelRequest(parts=[UserPromptPart(content='What is the capital of Italy?', timestamp=datetime.datetime(2026, 2, 21, 10, 16, 45, 903444, tzinfo=datetime.timezone.utc))], run_id='64dbfd38-2668-422c-a47c-d89de1647800'),
 ModelResponse(parts=[TextPart(content='The capital of Italy is **Rome**.')], usage=RequestUsage(input_tokens=14, output_tokens=10, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}), model_name='gpt-4.1-2025-04-14', timestamp=datetime.datetime(2026, 2, 21, 10, 16, 47, tzinfo=TzInfo(UTC)), provider_name='openai', provider_url='https://azure-m7byy3cl-eastus2.cognitiveservices.azure.com/openai/', provider_details={'finish_reason': 'stop'}, provider_response_id='chatcmpl-DBeKVBa3l4eTGYEuyL5fYYGLE3y4B', finish_reason='stop', run_id='64dbfd38-2668-422c-a47c-d89de1647800')]

**2. Stop on Structured Output Run**

In [6]:
from pydantic import BaseModel

class Person(BaseModel):
    """Defines a person"""
    name:str
    aget:str
    
events: list[AgentStreamEvent | AgentRunResultEvent] = []

model = create_pydantic_azure_openai(model_name = "gpt-4.1")

agent = Agent(
    model,
    output_type= str | Person,
    instructions = "You are a helpful assistant that answers questions about anything, but when the user metions a person, you will reply with a json object defined by the schema."

)


res = agent.run_sync("My name is Wang Huan, I am 32 years old, and I am a AI engineer living in Beijing.")
res

AgentRunResult(output=Person(name='Wang Huan', aget='32'))

In [7]:
res.all_messages()

[ModelRequest(parts=[UserPromptPart(content='My name is Wang Huan, I am 32 years old, and I am a AI engineer living in Beijing.', timestamp=datetime.datetime(2026, 2, 21, 10, 53, 51, 530826, tzinfo=datetime.timezone.utc))], instructions='You are a helpful assistant that answers questions about anything, but when the user metions a person, you will reply with a json object defined by the schema.', run_id='802401ca-5e4d-40e4-b0db-c51aa5ca3912'),
 ModelResponse(parts=[ToolCallPart(tool_name='final_result', args='{"name":"Wang Huan","aget":"32"}', tool_call_id='call_c9hodxOGVpujH6nNXLCKtZHN')], usage=RequestUsage(input_tokens=102, output_tokens=22, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}), model_name='gpt-4.1-2025-04-14', timestamp=datetime.datetime(2026, 2, 21, 10, 53, 52, tzinfo=TzInfo(UTC)), provider_name='openai', provider_url='https://azure-m7byy3cl-eastus2.cognitiveservices.azure.com/openai/', provider_detai